## In-memory montaging using bigfeta

First download and unzip an example directory.  This example uses:
21617_R1_166_T5_15_004637_20201231141040

autoTEM acquisitions create jsons of tile metadata including automatically derived lens corrections and point correspondences from template matching.

In [21]:
import copy
import os

import numpy
import pathlib

# renderapi is on pypi as render-python.  Models and interfaces with Render for alignment metadata
import renderapi

# bigfeta alignment solver
import bigfeta
import bigfeta.bigfeta
import bigfeta.utils


In [22]:
montage_dir = "/data/21617_R1_166_T5_15_004637_20201231141040/"
montage_dir_path = pathlib.Path(montage_dir)

In [29]:
# read tile metadata as render-python object
resolvedtiles_bn = "resolvedtiles_input.json.gz"
resolvedtiles_path = montage_dir_path / resolvedtiles_bn

input_resolvedtiles_json = bigfeta.utils.jsongz.load(str(resolvedtiles_path))
input_resolvedtiles = renderapi.resolvedtiles.ResolvedTiles(json=input_resolvedtiles_json)

In [30]:
# read point correspondences
correspondences_bn = "collection.json.gz"
correspondences_path = montage_dir_path / correspondences_bn

input_correspondences = bigfeta.utils.jsongz.load(str(correspondences_path))

### Solve for Affine Montage

In [73]:
# solve for affine montage using specific parameters

# BigFeta v1 can modify correspondences when building A -- copy to allow re-running
matches = copy.deepcopy(input_correspondences)

transform_name = "affine" # solve for affine transformation
transform_apply = []  # no additional transforms to apply to describe point correspondence conditions

# regularization dict with defined parameters.
#   It can be necessary to sweep across these parameters to find an optimal
#   configuration based on the montage and correspondence characteristics
regularization_dict = {
    "translation_factor": 1e-6,
    "default_lambda": 1e4
}


matrix_assembly_dict = {
    "npts_min": 5,  # minimum correspondences between tiles to consider
    "npts_max": 500,  # maximum correspondences between tiles
    "choose_random": False,  # do not randomly sample from correspondences when reducing to maximum points
    "depth": [0],  # montage stitching considers intra-section matches
    "inverse_dz": True,  # parameter not used in stitching -- whether to reduce weights as a function of inter-section pair distance
    "cross_pt_weight": 0.5,  # parameter not used in stitching -- base weight for inter-section matches
    "explicit_weight_by_depth": None,  # parameter not used in stitching -- specific weights given inter-section distances
    "montage_pt_weight": 1.0  # base weight for intra-section matches
}

create_CSR_A_args = (
    input_resolvedtiles, matches, transform_name,
    transform_apply, regularization_dict,
    matrix_assembly_dict
)


In [74]:
# create arrays used for solving system and tile object for output
fr, draft_resolvedtiles = bigfeta.bigfeta.create_CSR_A_fromobjects(
    *create_CSR_A_args, return_draft_resolvedtiles=True)

In [75]:
# create output resolvedtiles based on draft resolvedtiles (this is probably unnecessary)
output_rts = copy.deepcopy(draft_resolvedtiles)

solution_d = bigfeta.solve.solve(
    fr["A"], fr["weights"], fr["reg"], fr["x"], fr["rhs"])

# apply this result to the output resolvedtiles
bigfeta.utils.update_tilespecs(output_rts, solution_d["x"])


In [76]:
# show residuals, scale stdev, and scale mean
err_means = [e[0] for e in solution_d["err"]]
err_stds = [e[1] for e in solution_d["err"]]
errors = numpy.array(solution_d["error"]) / len(output_rts.tilespecs)

scales = numpy.array([ts.tforms[-1].scale
                          for ts in output_rts.tilespecs])
scale_mean = numpy.mean(scales, axis=0)
scale_stdevs = numpy.std(scales, axis=0)

print(f"scale means: {scale_mean}")
print(f"scale stdev: {scale_stdevs}")
print(f"errors: {errors / scale_mean}")

scale means: [1.0066988  1.02893452]
scale stdev: [0.00378605 0.00481382]
errors: [0.19593472 0.1246063 ]


Note that these correspondences taken as part of the autoTEM acquisition are selected from less deformed regions, so there are likely higher residuals (errors) than reported above at the e.g. the corners of the tiles.

### Next Steps:
 - write to render service using renderapi.resolvedtiles.put_tilespecs
 - view (using e.g. neuroglancer) or materialize